# Python: 你不知道的 super 

## super() 的入门使用

### 子类同时实现父类方法

**在类的继承中，子类修改父类同名方法，同时实现父类方法，但不覆盖父类方法**

http://python.jobbole.com/86787/

在类的继承中，如果重定义某个方法，该方法会覆盖父类的同名方法，但有时，我们希望能同时**实现父类的功能**，这时，我们就需要调用父类的方法了，可通过使用 super 来实现，比如：

In [14]:
class Animal(object):
    def __init__(self,name):
        self.name = name
    def greet(self):
        print('Hello,{}'.format(self.name))
        
class Dog(Animal):
    def greet(self):
        print('WangWang...')
        super(Dog, self).greet()  #python2 super().greet()

dog = Dog('dog')
dog.greet()

WangWang...
Hello,dog


在上面，Animal 是父类，Dog 是子类，我们在 Dog 类重定义了 greet 方法，为了能同时实现父类的功能，我们在子类中调用了父类的方法

### 子类同时拥有父类的属性

**在类的继承中，子类修改初始化方法，同时拥有父类的属性，但不覆盖父类属性**

为了同时拥有父类的属性，super 的一个最常见用法可以说是在子类中调用父类的初始化方法了。

In [2]:
class Base(object):
    def __init__(self,a,b):
        self.a = a
        self.b = b
        print('Base:a{},b{}'.format(self.a,self.b))
        
class A(Base):
    def __init__(self,a,b,c):
        super().__init__(a,b)  # Python2 super(A, self).__init__(a, b)
        #Base.__init__(self,a,b)
        self.c = c
        print('A:a{},b{},c{}'.format(self.a,self.b,self.c))

test = A(1,2,3)

print(A.mro())

Base:a1,b2
A:a1,b2,c3
[<class '__main__.A'>, <class '__main__.Base'>, <class 'object'>]


## 深入 super()

看了上面的使用，你可能会觉得 super 的使用很简单，同时实现父类的功能以及同时拥有父类的属性。

其实，在上面的情况下，super 获得的类刚好是父类，但在其他情况就不一定了，super 其实和父类没有实质性的关联。

让我们看一个稍微复杂的例子，涉及到多重继承，代码如下：

In [18]:
class Base(object):
    def __init__(self):
        print('enter Base')
        print('leave Base')
        
class A(Base):
    def __init__(self):
        print('enter A')
        super().__init__()
        print('leave A')
        
class B(Base):
    def __init__(self):
        print('enter B')
        super().__init__()
        print('leave B')
        
class C(A,B):
    def __init__(self):
        print('enter C')
        super().__init__()
        print('leave C')
        
a = A()

print('---------------------')
b = B()

print('---------------------')
c = C()

enter A
enter Base
leave Base
leave A
---------------------
enter B
enter Base
leave Base
leave B
---------------------
enter C
enter A
enter B
enter Base
leave Base
leave B
leave A
leave C


c = C() 预测的结果是

enter C
enter A
enter Base
leave Base
leave A
enter B
enter Base
leave Base
leave B
leave C

如果你认为 super 代表『调用父类的方法』，那你很可能会疑惑为什么 enter A 的下一句不是 enter Base 而是 enter B。原因是，super 和父类没有实质性的关联，现在让我们搞清 super 是怎么运作的。

## MRO 列表(Method Resolution Order)

事实上，对于你定义的每一个类，Python 会计算出一个**方法解析顺序（Method Resolution Order, MRO）列表**，它代表了类继承的顺序，我们可以使用下面的方式获得某个类的 MRO 列表：

```
类名.mro()
类名.__mro__
实例.__class__.mro()
```

In [21]:
print(C.mro())
print(C.__mro__)
print(C().__class__.mro())

[<class '__main__.C'>, <class '__main__.A'>, <class '__main__.B'>, <class '__main__.Base'>, <class 'object'>]
(<class '__main__.C'>, <class '__main__.A'>, <class '__main__.B'>, <class '__main__.Base'>, <class 'object'>)
enter C
enter A
enter B
enter Base
leave Base
leave B
leave A
leave C
[<class '__main__.C'>, <class '__main__.A'>, <class '__main__.B'>, <class '__main__.Base'>, <class 'object'>]


那这个 MRO 列表的顺序是怎么定的呢，它是通过一个 C3 线性化算法来实现的，这里我们就不去深究这个算法了，感兴趣的读者可以自己去了解一下，总的来说，一个类的 MRO 列表就是合并所有父类的 MRO 列表，并遵循以下三条原则：

- 子类永远在父类前面
- 如果有多个父类，会根据它们在列表中的顺序被检查
- 如果对下一个类存在两个合法的选择，选择第一个父类

## super 原理

super 的工作原理如下

In [ ]:
def super(cls, inst):
    mro = inst.__class__.mro()
    return mro[mro.index(cls) + 1]

其中，cls 代表类，inst 代表实例，上面的代码做了两件事：

- 获取 inst 的 MRO 列表
- 查找 cls 在当前 MRO 列表中的 index, 并返回它的下一个类，即 mro[index + 1]

当你使用 super(cls, inst) 时，Python 会在 inst 的 MRO 列表上搜索 cls 的下一个类。

现在，让我们回到前面的例子。

首先看类 C 的 __init__ 方法：`super(C, self).__init__()`

根据super()函数，
```
def super(cls,inst):
    mro = inst.__class__.mro()
    return mro[mro.index(cls)+1]
```
其中`super(C, self)`会返回 C.mro() 列表中后面一位，即`__main__.A`, 就变成`类A的 __init__()`，这时打印出 enter A，并执行下一行 `super(A, self).__init__()`

**注意：这里的 self 也是当前 C 的实例，MRO 列表跟上面是一样的**

搜索 A 在 MRO 中的下一个类，发现是 B，于是，跳到了`类B的__init__()`，这时会打印出 enter B，而不是 enter Base。并执行下一行 `super(B, self).__init__()`

同样，下一个类是 Base，跳到了`类Base的__init__()`，打印 enter Base 和 leave Base。

`类Base.__init__()`执行完，跳回到类B中的，执行`super(B, self).__init__()`下面一句，即打印 leave B。

`类B的__init__()`执行完，跳回到类A中的，执行`super(A, self).__init__()`下面一句，即打印 leave A。

`类A的__init__()`执行完，跳回到类C中的，执行`super(C, self).__init__()`下面一句，即打印 leave C。

整个过程还是比较清晰的，关键是要理解 super 的工作方式，而不是想当然地认为 super 调用了父类的方法。

## 小结

- 事实上，super 和父类没有实质性的关联。

- super(cls, inst) 获得的是 cls 在 inst 的 MRO 列表中的下一个类。

In [ ]:
class Base(object):
    def __init__(self):
        print('enter Base')
        print('leave Base')
        
class A(Base):
    def __init__(self):
        print('enter A')
        super().__init__()
        print('leave A')
        
class B(Base):
    def __init__(self):
        print('enter B')
        super().__init__()
        print('leave B')
        
class C(A,B):
    def __init__(self):
        print('enter C')
        super().__init__()
        print('leave C')
        
a = A()

print('---------------------')
b = B()

print('---------------------')
c = C()